In [1]:
import os
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import balanced_accuracy_score
from _CNN.CNN import CNNClassifer  # importa la tua rete

# ======================================================
# FUNZIONE DI OTTIMIZZAZIONE
# ======================================================
def objective(trial):
    data_root = "/Users/saracurti/Desktop/images_split_and_augmented"  
    num_classes = 3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # === Parametri da ottimizzare ===
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.6)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "SGD"])

    # === Trasformazioni (niente augmentation) ===
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3304, 0.2551, 0.2663],
                             std=[0.3395, 0.2703, 0.2817])
    ])

    train_data = datasets.ImageFolder(os.path.join(data_root, "train"), transform=transform)
    val_data = datasets.ImageFolder(os.path.join(data_root, "val"), transform=transform)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

    # === Modello ===
    model = CNNClassifer(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()

    # === Ottimizzatore ===
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

    # === Training breve (solo 5 epoche per velocità) ===
    model.train()
    for epoch in range(5):
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # === Validazione ===
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    bal_acc = balanced_accuracy_score(y_true, y_pred)
    return bal_acc  # metrica da massimizzare




/Users/saracurti/myproject/Colposcopy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # 🔹 aumenta per maggiore accuratezza

print("\n=== 🔍 MIGLIORI IPERPARAMETRI TROVATI ===")
for key, value in study.best_params.items():
    print(f"{key}: {value}")
print(f"Balanced Accuracy: {study.best_value:.4f}")


[I 2025-11-10 00:32:34,239] A new study created in memory with name: no-name-4af2dbac-0319-4c90-a2f9-7a1c4ba3f759
/var/folders/r6/jyz2xchn3dn7rp4vqxgd0kw80000gn/T/ipykernel_60627/1838881912.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
/var/folders/r6/jyz2xchn3dn7rp4vqxgd0kw80000gn/T/ipykernel_60627/1838881912.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.6)
/var/folders/r6/jyz2xchn3dn7rp4vqxgd0kw80000gn/T/ipykernel_60627/1838881912.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/op


=== 🔍 MIGLIORI IPERPARAMETRI TROVATI ===
lr: 0.001795334115473034
batch_size: 16
dropout_rate: 0.4010294331533155
weight_decay: 6.592817108874535e-05
optimizer: SGD
Balanced Accuracy: 0.4771
